In [1]:
import tensorflow as tf
import numpy as np
import random
import os
import sys
import tqdm
import csv
from PIL import Image
import cv2
from matplotlib import pyplot as plt

In [2]:
def image_load_and_batch(img_list):
    im = []
    
    for img_name in img_list:
        
        img =cv2.imread(img_name)
        b, g, r = cv2.split(img)
        im.append(cv2.merge([r,g,b]))

            
    return im

In [3]:
def label_make_batch(label_list):
    output = []
    
    for label in label_list:
        
        x = label[1]
        y = label[0]
        if x == None or y == None:
            
            output.append(np.concatenate((np.ones((268,480,1)),np.zeros((268,480,1))), axis=2))
            
        else:

            x = x//4
            y = y//4
            ball_map1 = np.ones((268,480,1))
            ball_map1[x-2:x+2,y-2:y+2] = 0
            ball_map2 = np.zeros((268,480,1))
            ball_map2[x-2:x+2,y-2:y+2] = 1
            last = np.concatenate((ball_map1,ball_map2),axis=2)
            
            output.append(last)

    
    return output
            

In [4]:
def make_batch(all_data):
    
    img_list = []
    label_list = []
    
    keys = list(all_data.keys())
    random.shuffle(keys)
    for key in keys:
        img_list.append(key)
        label_list.append(all_data[key])
    
    train_img = img_list[:8000]
    test_img = img_list[8000:]
    
    train_label = label_list[:8000]
    test_label = label_list[8000:]
    
    return train_img, test_img, train_label, test_label

In [ ]:
def data argumentation(argu_keys, data, label):
    
    if argu_keys == "fr_ud":
        return tf.image.flip_up_down(data),tf.image.flip_up_down(label)
    
    elif argu_keys == "fr_lr":
        return tf.image.flip_left_right(data),tf.image.flip_left_right(label)
    
    elif argu_keys == "ro_90":
        return tf.image.rot90(data, k=1), tf.image.rot90(data, k=1)
    
    elif argu_keys == "ro_180":
        return tf.image.rot90(data, k=2), tf.image.rot90(data, k=2)
    
    elif argu_keys == "Gau":
        noise = tf.random_normal(shape=tf.shape(data), mean=0.0, stddev=1.0,dtype=tf.float32)
        return tf.add(data,noise), label
    else:
        return data,label
        

In [5]:
def Conv_block(data, train=True, filter1=3, filter2=3, maxpool=True, std =1, in_ch=3, out_ch=3):    
    
    
    conv1 = tf.layers.conv2d(data, filters=out_ch,kernel_size=filter1, strides=std, padding="SAME", trainable=train)
    output = tf.layers.conv2d(conv1, filters=out_ch,kernel_size=filter2, strides=1, padding='SAME',trainable=train)
    if maxpool:
        output = tf.layers.max_pooling2d(inputs = output, pool_size=2, strides=2, padding='VALID')
    
    return output

In [6]:
def Upsample_block(data, output_size, kernel=2, std=2):
    
    output = tf.layers.conv2d_transpose(data, filters=output_size, kernel_size=kernel, strides=std, padding="VALID")
    
    return output

In [7]:
def Model(data, label, train, batch_size):

    
    
    with tf.variable_scope("conv1"):
        Conv1_feature = Conv_block(data, train, filter1=7, filter2=3, maxpool=True, std=2,in_ch=3,out_ch=8)
        Conv1_feature = tf.slice(Conv1_feature,[0,0,0,0],[batch_size,268,480,8])
        print("Conv1 Done")

    with tf.variable_scope("conv2"):
        Conv2_feature = Conv_block(Conv1_feature, train, filter1=7, filter2=3, maxpool=True, in_ch=8,out_ch=16)
        print("Conv2 Done")
    with tf.variable_scope("conv3"):
        Conv3_feature = Conv_block(Conv2_feature, train, filter1=7, filter2=3, maxpool=True, in_ch=16,out_ch=32)
        print("Conv3 Done")
        
     
    
    upsize = Conv1_feature.get_shape().as_list()[-1]      
    with tf.variable_scope("Upsize1"):
        Conv2_up = Upsample_block(Conv2_feature, upsize, kernel=2, std=2)        
        print("Upsize1 Done")
    with tf.variable_scope("Upsize2"):
        upsize2 = Conv2_feature.get_shape().as_list()[-1]
        Conv3_up = Upsample_block(Conv3_feature, upsize, kernel=4, std=4)
        print("Upsize2 Done")
        '''bias = tf.zeros([batch_size, 2, 480, 8])
        Conv3_up = tf.concat([Conv3_up, bias], axis=1)'''

        
        
    with tf.variable_scope("Concat_conv"):
        Conv_concat = tf.concat([Conv1_feature, Conv2_up, Conv3_up],axis=3)  
        print("Concat Done")
    with tf.variable_scope("conv4"):
        Conv4_feature = Conv_block(Conv_concat, train, filter1=7, filter2=3, maxpool=False, in_ch = 56,out_ch=2)
        print("Conv4 Done")
        #print(Conv4_feature)


    return Conv4_feature

In [10]:
def optimizer(data, label, lr=0.0001):
    #print(data)
    logits = tf.nn.softmax(data)
    print("lgits done")
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=data, labels=label)
    print("entropy done")
    loss_op = tf.reduce_mean(cross_entropy, name="Loss")
    print("loss Done")
    train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss_op, name="Train_op")
    print("train Done")
    
    return logits, loss_op, train_op
    

In [8]:
# Data Load Part
# Data Annotate In Deep_Ball_Data/Annote/ and image Deep_Ball_Data/img


#Annotate Data Load
f = open("./Deep_Ball_Data/Annote/cam.csv", 'r', encoding='utf-8')
k = 4
rdr = csv.reader(f)
Data = []
for line in rdr:
    if line[0][0] == 'F':continue
    if line[1] == ' -': 
        Data.append([None, None])
    else:
        x = int(line[1])
        y = int(line[2])
        Data.append([x,y])
                    
f.close()

#Image Data Load

path = "./Deep_Ball_Data/img/graz-arbesbach_5"
img_list = [os.path.join(path , x) for x in os.listdir(path)]


#merge all data Want to shuffle
all_data = {}
for i,img in enumerate(img_list[:9000]):
    
    all_data[img]= Data[i]

In [9]:
#Test_data make_file
file_path = "./Deep_Ball_Result/"
f=open(file_path + "Test.txt", 'w', encoding='utf-8')
for i in img_list[9000:]:
    f.write(i)
    f.write("\n")
f.close()


In [11]:
n_epoch = 70
batch = 4000
batch_size = 16
lr = 0.001
total_loss = 0
train=True
argu_keys = ['fr_ud','fr_lr','ro_90','ro_180','Gau']

tf.reset_default_graph()


input_image = tf.placeholder(tf.float32,[None, 1080, 1920, 3])
label = tf.placeholder(tf.float32,[None, 268, 480, 2])
init = tf.global_variables_initializer()


with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)), log_device_placement=True)) as sess:
    
    
    output = Model(input_image, label, train, batch_size)
    
    logits, loss, train_op = optimizer(output, label, lr)
    
    
    saver = tf.train.Saver(max_to_keep = 8000000)
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    print("Training.. Start..")
    
    for epoch in range(n_epoch):
        counter = 0
        batch = 0
        total_loss = 0
        total_acc = 0
        Train_image, Test_image, Train_label, Test_label = make_batch(all_data)

        while True:
            print("train epoch", epoch, "train batch :", batch,"~",batch +batch_size)
            
            train=True


            now_img = image_load_and_batch(Train_image[batch:batch +batch_size])
            now_label = label_make_batch(Train_label[batch:batch +batch_size])
            now_total_pixcel = np.shape(now_label)[0] *np.shape(now_label)[1] *np.shape(now_label)[2]* np.shape(now_label)[3]
           
            try:
                logit, losses, _ = sess.run([logits, loss, train_op], feed_dict={input_image:now_img, label:now_label})
                counter += 1
                pred = np.argmax(logit, axis=3)
                truth = np.argmax(now_label, axis =3)
                now_acc= (1-(np.shape(np.nonzero(pred-truth))[-1]/now_total_pixcel)) * 100
                print(now_acc)
                total_loss += losses
                total_acc += now_acc
                
            except TypeError:
                pass
            
            #print(type(len(np.nonzero(logit - np.array(now_label)))))
            
            
           
            

            batch = batch+batch_size
           
                
            if batch==4000 or batch==8000:
                
                print("Now...",str(epoch),"... ",str(batch) ,"..End..")
                print("Now Loss..", str(total_loss), " Now Accuracy..", str(total_acc / counter))
                print("Validation Test Start..")
                
                counter =0
                test_batch = 0
                total_loss = 0
                total_acc = 0
                train = False
                
                while True:
                    
                    print("test_batch : ", test_batch,"~",test_batch + batch_size)
                    test_img = image_load_and_batch(Test_image[test_batch:test_batch+ batch_size])
                    test_label = label_make_batch(Test_label[test_batch:test_batch+ batch_size])
                    #print(np.shape(test_img))
                    #print(np.shape(test_label))
                    
                    try:
                        
                        test_logit, test_losses, test_train_op = sess.run([logits, loss, train_op], feed_dict={input_image:test_img, label:test_label})
                       
                        pred = np.argmax(test_logit, axis=3)
                        truth = np.argmax(test_label, axis =3)
                        now_acc= (1-(np.shape(np.nonzero(pred-truth))[-1]/now_total_pixcel)) * 100
                        
                        print(now_acc)
                        counter += 1
                        test_batch += batch_size
                        total_acc += now_acc
                        total_loss += losses


                        if test_batch>991:

                            print("Validation Test End..")
                            print("Now Loss..", str(total_loss), "Now Accuracy..", str(total_acc / counter))
                            saver.save(sess, './Deep_Ball_Result/Deep_Ball_Model_' + str(epoch)+'_'+str(batch) + '.ckpt')
                            break
                    
                    except TypeError:
                        test_batch += batch_size
                        if test_batch>991:

                            print("Validation Test End..")
                            print("Now Loss..", str(total_loss), "Now Accuracy..", str(total_acc / counter))
                            saver.save(sess, './Deep_Ball_Result/Deep_Ball_Model_' + str(epoch)+'_'+str(batch) + '.ckpt')
                            break
                        continue
                    
                    
                    
                    
                    
                
                
                
            if batch>7999: break
                
            


Conv1 Done
Conv2 Done
Conv3 Done
Upsize1 Done
Upsize2 Done
Concat Done
Conv4 Done
lgits done
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

entropy done
loss Done
train Done
Training.. Start..
train epoch 0 train batch : 0 ~ 16
99.99650186567163
train epoch 0 train batch : 16 ~ 32
99.15294134794776
train epoch 0 train batch : 32 ~ 48
99.99399972792288
train epoch 0 train batch : 48 ~ 64
99.99778937344527
train epoch 0 train batch : 64 ~ 80
99.99759503264926
train epoch 0 train batch : 80 ~ 96
99.99650186567163
train epoch 0 train batch : 96 ~ 112
99.99844527363184
train epoch 0 train batch : 112 ~ 128
99.99727922885572
train epoch 0 train batch : 128 ~ 144
99.99766791044776
train epoch 0 train batch : 144 ~ 160
99.9980565920398
train epoch 0 train batch : 160 ~ 176
99.99766791044776
train epoch 0 train batch : 176 ~ 192
99.99727922885572
train 

test_batch :  544 ~ 560
99.9931980721393
test_batch :  560 ~ 576
99.99302802394278
test_batch :  576 ~ 592
99.9912789567786
test_batch :  592 ~ 608
99.9902100824005
test_batch :  608 ~ 624
99.98880111162936
test_batch :  624 ~ 640
99.9930037313433
test_batch :  640 ~ 656
99.99424265391791
test_batch :  656 ~ 672
99.99555445429105
test_batch :  672 ~ 688
99.99548157649254
test_batch :  688 ~ 704
99.99635611007463
test_batch :  704 ~ 720
99.99540869869404
test_batch :  720 ~ 736
99.99458275031094
test_batch :  736 ~ 752
99.99285797574626
test_batch :  752 ~ 768
99.990185789801
test_batch :  768 ~ 784
99.99159476057214
test_batch :  784 ~ 800
99.99635611007463
test_batch :  800 ~ 816
99.99570020988806
test_batch :  816 ~ 832
99.99453416511194
test_batch :  832 ~ 848
99.99635611007463
test_batch :  848 ~ 864
99.99358675373135
test_batch :  864 ~ 880
99.99222636815921
test_batch :  880 ~ 896
99.98948130441542
test_batch :  896 ~ 912
99.99365963152985
test_batch :  912 ~ 928
99.9941211909204

99.99472850590796
test_batch :  224 ~ 240
99.99553016169153
test_batch :  240 ~ 256
99.99548157649254
test_batch :  256 ~ 272
99.9951414800995
test_batch :  272 ~ 288
99.99480138370646
test_batch :  288 ~ 304
99.99718205845771
test_batch :  304 ~ 320
99.99786225124379
test_batch :  320 ~ 336
99.99732781405473
test_batch :  336 ~ 352
99.99650186567163
test_batch :  352 ~ 368
99.99467992070895
test_batch :  368 ~ 384
99.99579738028606
test_batch :  384 ~ 400
99.99373250932835
test_batch :  400 ~ 416
99.99686625466417
test_batch :  416 ~ 432
99.99664762126865
test_batch :  432 ~ 448
99.99613747667911
test_batch :  448 ~ 464
99.99613747667911
test_batch :  464 ~ 480
99.9774807602612
test_batch :  480 ~ 496
99.99783795864428
test_batch :  496 ~ 512
99.99759503264926
test_batch :  512 ~ 528
99.9980080068408
test_batch :  528 ~ 544
99.9961131840796
test_batch :  544 ~ 560
99.99839668843283
test_batch :  560 ~ 576
99.99689054726369
test_batch :  576 ~ 592
99.99723064365672
test_batch :  592 ~ 

train epoch 1 train batch : 3984 ~ 4000
99.9970848880597
Now... 1 ...  4000 ..End..
Now Loss.. 1.0864511700347066  Now Accuracy.. 99.99540529773014
Validation Test Start..
test_batch :  0 ~ 16
99.9951414800995
test_batch :  16 ~ 32
99.99740069185323
test_batch :  32 ~ 48
99.99582167288558
test_batch :  48 ~ 64
99.99664762126865
test_batch :  64 ~ 80
99.99689054726369
test_batch :  80 ~ 96
99.9979594216418
test_batch :  96 ~ 112
99.99793512904229
test_batch :  112 ~ 128
99.99689054726369
test_batch :  128 ~ 144
99.99621035447761
test_batch :  144 ~ 160
99.99740069185323
test_batch :  160 ~ 176
99.99642898787313
test_batch :  176 ~ 192
99.99776508084577
test_batch :  192 ~ 208
99.99691483986318
test_batch :  208 ~ 224
99.99757074004975
test_batch :  224 ~ 240
99.99749786225124
test_batch :  240 ~ 256
99.99742498445273
test_batch :  256 ~ 272
99.99740069185323
test_batch :  272 ~ 288
99.99664762126865
test_batch :  288 ~ 304
99.9951171875
test_batch :  304 ~ 320
99.99567591728857
test_bat

train epoch 1 train batch : 7728 ~ 7744
99.99669620646766
train epoch 1 train batch : 7744 ~ 7760
99.99720635105722
train epoch 1 train batch : 7760 ~ 7776
99.99715776585822
train epoch 1 train batch : 7776 ~ 7792
99.99652615827115
train epoch 1 train batch : 7792 ~ 7808
99.99625893967662
train epoch 1 train batch : 7808 ~ 7824
99.99703630286069
train epoch 1 train batch : 7824 ~ 7840
99.99676908426616
train epoch 1 train batch : 7840 ~ 7856
99.99635611007463
train epoch 1 train batch : 7856 ~ 7872
99.99596742848259
train epoch 1 train batch : 7872 ~ 7888
99.99390255752488
train epoch 1 train batch : 7888 ~ 7904
99.994971431903
train epoch 1 train batch : 7904 ~ 7920
99.99560303949005
train epoch 1 train batch : 7920 ~ 7936
99.99839668843283
train epoch 1 train batch : 7936 ~ 7952
99.99774078824628
train epoch 1 train batch : 7952 ~ 7968
99.99866390702736
train epoch 1 train batch : 7968 ~ 7984
99.99735210665422
train epoch 1 train batch : 7984 ~ 8000
99.99696342506219
Now... 1 ...  80

...

train epoch 68 train batch : 16 ~ 32
99.99689054726369
train epoch 68 train batch : 32 ~ 48
99.99766791044776
train epoch 68 train batch : 48 ~ 64
99.99727922885572
train epoch 68 train batch : 64 ~ 80
99.9980565920398
train epoch 68 train batch : 80 ~ 96
99.99727922885572
train epoch 68 train batch : 96 ~ 112
99.99689054726369
train epoch 68 train batch : 112 ~ 128
99.99844527363184
train epoch 68 train batch : 128 ~ 144
99.99727922885572
train epoch 68 train batch : 144 ~ 160
99.99727922885572
train epoch 68 train batch : 160 ~ 176
99.99766791044776
train epoch 68 train batch : 176 ~ 192
99.99883395522387
train epoch 68 train batch : 192 ~ 208
99.99727922885572
train epoch 68 train batch : 208 ~ 224
99.9980565920398
train epoch 68 train batch : 224 ~ 240
99.99689054726369
train epoch 68 train batch : 240 ~ 256
99.9980565920398
train epoch 68 train batch : 256 ~ 272
99.99727922885572
train epoch 68 train batch : 272 ~ 288
99.99844527363184
train epoch 68 train batch : 288 ~ 304
99.996

99.99689054726369
test_batch :  672 ~ 688
99.99572450248756
test_batch :  688 ~ 704
99.99766791044776
test_batch :  704 ~ 720
99.99844527363184
test_batch :  720 ~ 736
99.9980565920398
test_batch :  736 ~ 752
99.99650186567163
test_batch :  752 ~ 768
99.9980565920398
test_batch :  768 ~ 784
99.99572450248756
test_batch :  784 ~ 800
99.99727922885572
test_batch :  800 ~ 816
99.99766791044776
test_batch :  816 ~ 832
99.99689054726369
test_batch :  832 ~ 848
99.99883395522387
test_batch :  848 ~ 864
99.99766791044776
test_batch :  864 ~ 880
99.99572450248756
test_batch :  880 ~ 896
99.99689054726369
test_batch :  896 ~ 912
99.99689054726369
test_batch :  912 ~ 928
99.99883395522387
test_batch :  928 ~ 944
99.99727922885572
test_batch :  944 ~ 960
99.99689054726369
test_batch :  960 ~ 976
99.9980565920398
test_batch :  976 ~ 992
99.99689054726369
Validation Test End..
Now Loss.. 659995.630859375 Now Accuracy.. 99.99745476247789
train epoch 68 train batch : 4000 ~ 4016
99.9980565920398
trai

99.9961131840796
test_batch :  240 ~ 256
99.9980565920398
test_batch :  256 ~ 272
99.9961131840796
test_batch :  272 ~ 288
99.99844527363184
test_batch :  288 ~ 304
99.99689054726369
test_batch :  304 ~ 320
99.99727922885572
test_batch :  320 ~ 336
99.9961131840796
test_batch :  336 ~ 352
99.99727922885572
test_batch :  352 ~ 368
99.99844527363184
test_batch :  368 ~ 384
99.99727922885572
test_batch :  384 ~ 400
99.99689054726369
test_batch :  400 ~ 416
99.99689054726369
test_batch :  416 ~ 432
99.99572450248756
test_batch :  432 ~ 448
99.99650186567163
test_batch :  448 ~ 464
99.9961131840796
test_batch :  464 ~ 480
99.99533582089552
test_batch :  480 ~ 496
99.99650186567163
test_batch :  496 ~ 512
99.99650186567163
test_batch :  512 ~ 528
99.99533582089552
test_batch :  528 ~ 544
99.99572450248756
test_batch :  544 ~ 560
99.99650186567163
test_batch :  560 ~ 576
99.99689054726369
test_batch :  576 ~ 592
99.99689054726369
test_batch :  592 ~ 608
99.99689054726369
test_batch :  608 ~ 6

train epoch 69 train batch : 3920 ~ 3936
99.995020017102
train epoch 69 train batch : 3936 ~ 3952
99.99436411691542
train epoch 69 train batch : 3952 ~ 3968
99.99543299129353
train epoch 69 train batch : 3968 ~ 3984
99.994971431903
train epoch 69 train batch : 3984 ~ 4000
99.99421836131842
Now... 69 ...  4000 ..End..
Now Loss.. 143235.65238952637  Now Accuracy.. 99.991397699005
Validation Test Start..
test_batch :  0 ~ 16
99.99276080534825
test_batch :  16 ~ 32
99.96754508706468
test_batch :  32 ~ 48
99.99460704291045
test_batch :  48 ~ 64
99.99526294309702
test_batch :  64 ~ 80
99.99370821672886
test_batch :  80 ~ 96
99.99346529073382
test_batch :  96 ~ 112
99.9950928949005
test_batch :  112 ~ 128
99.99540869869404
test_batch :  128 ~ 144
99.99431553171641
test_batch :  144 ~ 160
99.99562733208955
test_batch :  160 ~ 176
99.99399972792288
test_batch :  176 ~ 192
99.99375680192786
test_batch :  192 ~ 208
99.99203202736318
test_batch :  208 ~ 224
99.94400555814677
test_batch :  224 ~ 24

train epoch 69 train batch : 7616 ~ 7632
99.9922749533582
train epoch 69 train batch : 7632 ~ 7648
99.99188627176618
train epoch 69 train batch : 7648 ~ 7664
99.9923478311567
train epoch 69 train batch : 7664 ~ 7680
99.99042871579601
train epoch 69 train batch : 7680 ~ 7696
99.99157046797264
train epoch 69 train batch : 7696 ~ 7712
99.98884969682837
train epoch 69 train batch : 7712 ~ 7728
99.99079310478855
train epoch 69 train batch : 7728 ~ 7744
99.99079310478855
train epoch 69 train batch : 7744 ~ 7760
99.990137204602
train epoch 69 train batch : 7760 ~ 7776
99.98846101523633
train epoch 69 train batch : 7776 ~ 7792
99.9882909670398
train epoch 69 train batch : 7792 ~ 7808
99.98593458488806
train epoch 69 train batch : 7808 ~ 7824
99.98515722170399
train epoch 69 train batch : 7824 ~ 7840
99.9893355488184
train epoch 69 train batch : 7840 ~ 7856
99.98748931125621
train epoch 69 train batch : 7856 ~ 7872
99.98999144900498
train epoch 69 train batch : 7872 ~ 7888
99.98894686722637
tra